### 引入依赖

In [ ]:
from autogen_core.models import ModelInfo
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
from pathlib import Path

### 获取mcp工具列表

In [42]:
# 获取当前工作目录
workdir = Path.cwd()
server_path = workdir.parent / "src" / "mcp-demo" / "server.py"
server_params = StdioServerParams(command="uv", args=["run", str(server_path)])
tools = await mcp_server_tools(server_params)

### 为agent分配工具

In [ ]:
model_info = ModelInfo(
    family="r1", vision=False, function_calling=True, json_output=True
)
model_client = OllamaChatCompletionClient(model="llama3.1:8b", model_info=model_info)


class MyAssistant:
    """此助手角色只能进行货币转换"""
    def __init__(self):
        self.role = "currency_convert_role"
        self.agent = AssistantAgent(
            name="assistant",
            model_client=model_client,
            tools=[tool for tool in tools if tool.name.startswith("currency")],
            system_message="你是一个乐于助人的助手，如果没有工具可调用，请回复：没有工具可调用。",
            reflect_on_tool_use=True,
            model_client_stream=True,  # Enable streaming tokens from the model client.
        )

### 开始任务

In [44]:
async def main() -> None:
    myAssistant = MyAssistant()
    await Console(myAssistant.agent.run_stream(task="杭州今天适合出游吗"), output_stats=True)
    await Console(myAssistant.agent.run_stream(task="使用最新汇率将34美元转换成人民币"), output_stats=True)
    # Close the connection to the model client.
    await model_client.close()


await main()

---------- TextMessage (user) ----------
杭州今天适合出游吗
---------- ModelClientStreamingChunkEvent (assistant) ----------
没有工具可调用，请重新提问。
[Prompt tokens: 194, Completion tokens: 0]
---------- ToolCallExecutionEvent (assistant) ----------
[]
---------- ModelClientStreamingChunkEvent (assistant) ----------
没有杭州市的天气预报信息，无法准确判断杭州今天是否适合出游。
[Prompt tokens: 49, Completion tokens: 25]
---------- Summary ----------
Number of messages: 4
Finish reason: None
Total prompt tokens: 243
Total completion tokens: 25
Duration: 2.89 seconds
---------- TextMessage (user) ----------
使用最新汇率将34美元转换成人民币
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='0', arguments='{"request": {"amount": 34, "from_currency": "USD", "to_currency": "CNY"}}', name='currency_converter')]
[Prompt tokens: 240, Completion tokens: 43]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='[{"type": "text", "text": "{\\n  \\"amount\\": 34.0,\\n  \\"converted\\": 245.82,\\n  \\"rate